In [10]:
from pathlib import Path
import pandas as pd
from sklearn.metrics import cohen_kappa_score

In [53]:


# GitHub Copilot
# Código para ler arquivos CSV separados por '|' em data/anotated/complete
# Coloca os DataFrames nas variáveis cezio, evandro, mateus e cria `combined`


base_dir = Path("inflação/data/anotated/complete")
filenames = ["cezio.csv", "evandro.csv", "mateus.csv"]

data = {}
for fname in filenames:
    path = base_dir / fname
    if not path.exists():
        print(f"Aviso: arquivo não encbontrado: {path}")
        continue
    try:
        df = pd.read_csv(path, sep="|", engine="python", encoding="utf-8")
    except Exception as e:
        print(f"Erro ao ler {path}: {e}")
        continue
    data[path.stem] = df

cezio = data.get("cezio")
evandro = data.get("evandro")
mateus = data.get("mateus")

combined = pd.concat([df for df in data.values()], ignore_index=True) if data else pd.DataFrame()

# Exemplo rápido de verificação
print({k: (v.shape if v is not None else None) for k, v in {"cezio": cezio, "evandro": evandro, "mateus": mateus}.items()})
print("combined shape:", combined.shape)

{'cezio': (99, 4), 'evandro': (125, 4), 'mateus': (120, 4)}
combined shape: (344, 4)


In [54]:
for fname in filenames:
    path = base_dir / fname
    if not path.exists():
        print(f"Aviso: arquivo não encbontrado: {path}")
        continue
    try:
        df = pd.read_csv(path, sep="|", engine="python", encoding="utf-8")
        # pegar apenas as primeiras 99 linhas de cada avaliador
        df = df.head(99).reset_index(drop=True)
    except Exception as e:
        print(f"Erro ao ler {path}: {e}")
        continue
    data[path.stem] = df

cezio = data.get("cezio")
evandro = data.get("evandro")
mateus = data.get("mateus")

evandro

,Date,Model,Grade,Sentence
0,02/09/2009,human,-1,"No médio prazo, o risco advém dos efeitos, cum..."
1,30/11/2011,human,-1,"Os preços livres aumentaram 7, 24% no período ..."
2,21/10/2009,human,-1,"Nesse contexto, após um período de flexibiliza..."
3,17/04/2013,human,-1,"Considerada a série observada, o Nuci apresent..."
4,20/11/2002,human,-1,Dados preliminares da Fecomércio mostram que o...
...,...,...,...,...
94,18/07/2007,human,-1,Dados compilados pela Fundação Getulio Vargas ...
95,22/07/2009,human,-3,Estimativas recentes do Fundo Monetário Intern...
96,19/06/2024,human,-1,O Comitê reforçou a visão de que o esmorecimen...
97,04/08/2021,human,1,"De acordo com o cenário básico, que utiliza a ..."


In [55]:


def pairwise_kappa(df1, df2, on='Sentence', grade_col='Grade', name1='a', name2='b'):
    m = pd.merge(df1[[on, grade_col]], df2[[on, grade_col]], on=on, how='inner',
                 suffixes=(f"_{name1}", f"_{name2}"))
    if m.empty:
        print(f"Aviso: sem correspondências entre {name1} e {name2} (coluna '{on}').")
        return None
    k = cohen_kappa_score(m[f"{grade_col}_{name1}"], m[f"{grade_col}_{name2}"])
    print(f"{name1} vs {name2}: Cohen's kappa = {k:.4f} (n = {len(m)})")
    return k, m

results = {}
results['cezio_vs_evandro'] = pairwise_kappa(cezio, evandro, name1='cezio', name2='evandro')
results['cezio_vs_mateus'] = pairwise_kappa(cezio, mateus, name1='cezio', name2='mateus')
results['evandro_vs_mateus'] = pairwise_kappa(evandro, mateus, name1='evandro', name2='mateus')

# resumo em DataFrame
summary = []
for pair, res in results.items():
    if res is None:
        summary.append({'pair': pair, 'kappa': None, 'n': 0})
    else:
        kappa, matched = res
        summary.append({'pair': pair, 'kappa': float(kappa), 'n': len(matched)})

pd.DataFrame(summary)

cezio vs evandro: Cohen's kappa = 0.3267 (n = 99)
cezio vs mateus: Cohen's kappa = 0.4079 (n = 99)
evandro vs mateus: Cohen's kappa = 0.3435 (n = 99)


,pair,kappa,n
0,cezio_vs_evandro,0.326707,99
1,cezio_vs_mateus,0.407945,99
2,evandro_vs_mateus,0.343518,99


In [56]:
# montar um DataFrame com uma coluna de avaliação por avaliador (usando 'Sentence' como chave)
cezio_g = cezio[['Sentence', 'Grade']].rename(columns={'Grade': 'Grade_cezio'})
evandro_g = evandro[['Sentence', 'Grade']].rename(columns={'Grade': 'Grade_evandro'})
mateus_g = mateus[['Sentence', 'Grade']].rename(columns={'Grade': 'Grade_mateus'})

# 1) Inner join: somente sentenças avaliadas pelos três avaliadores
evaluations = cezio_g.merge(evandro_g, on='Sentence', how='inner') \
                     .merge(mateus_g, on='Sentence', how='inner')

print("evaluations (inner) shape:", evaluations.shape)
evaluations.head()

# 2) (opcional) Outer join: todas as sentenças, NaN quando faltar avaliação
evaluations_all = cezio_g.merge(evandro_g, on='Sentence', how='outer') \
                        .merge(mateus_g, on='Sentence', how='outer')

print("evaluations_all (outer) shape:", evaluations_all.shape)
evaluations_all

evaluations (inner) shape: (99, 4)
evaluations_all (outer) shape: (99, 4)


,Sentence,Grade_cezio,Grade_evandro,Grade_mateus
0,A atividade econômica deverá se recuperar nos ...,1,1,1
1,A carteira de operações realizadas com recurso...,-2,-3,-2
2,"A conjuntura atual, caracterizada por um estág...",1,0,1
3,A decisão do Copom de reduzir a meta para a ta...,1,1,1
4,A elevação da meta para a Taxa Selic nas duas ...,-2,0,-2
...,...,...,...,...
94,Progressos na estrutura dos mercados financeir...,1,1,1
95,"Quanto às taxas de inflação, elas têm permanec...",-2,0,0
96,Relativamente ao Índice Nacional de Preços ao ...,0,0,-1
97,Segundo pesquisa do IBGE nas principais regiõe...,-2,-3,-2


In [57]:
for ev in evaluations_all:
    print(ev)

Sentence
Grade_cezio
Grade_evandro
Grade_mateus


In [61]:
# print rows from evaluations_all where any Grade_* is NaN
grade_cols = [c for c in evaluations_all.columns if c.lower().startswith('grade')]
for i, row in evaluations_all.iterrows():
    if row[grade_cols].isna().any():
        print(f"Row {i}:", row.to_dict())

In [62]:
# montar DataFrame com linhas onde pelo menos 2 avaliadores concordam (usando evaluations_all e grade_cols já existentes)
def _match_info(row):
    vc = row[grade_cols].value_counts()
    if len(vc) > 0 and vc.iloc[0] >= 2:
        matched_grade = vc.index[0]
        matched_by = [c for c in grade_cols if row[c] == matched_grade]
        return pd.Series({'matched_grade': int(matched_grade), 'matched_by': matched_by, 'match_count': int(vc.iloc[0])})
    return pd.Series({'matched_grade': None, 'matched_by': [], 'match_count': 0})

info = evaluations_all.apply(_match_info, axis=1)
consensus = pd.concat([evaluations_all, info], axis=1)
consensus = consensus[consensus['match_count'] >= 2].reset_index(drop=True)

print("consensus shape:", consensus.shape)
consensus.head()

consensus shape: (88, 7)


,Sentence,Grade_cezio,Grade_evandro,Grade_mateus,matched_grade,matched_by,match_count
0,A atividade econômica deverá se recuperar nos ...,1,1,1,1.0,"[Grade_cezio, Grade_evandro, Grade_mateus]",3
1,A carteira de operações realizadas com recurso...,-2,-3,-2,-2.0,"[Grade_cezio, Grade_mateus]",2
2,"A conjuntura atual, caracterizada por um estág...",1,0,1,1.0,"[Grade_cezio, Grade_mateus]",2
3,A decisão do Copom de reduzir a meta para a ta...,1,1,1,1.0,"[Grade_cezio, Grade_evandro, Grade_mateus]",3
4,A elevação da meta para a Taxa Selic nas duas ...,-2,0,-2,-2.0,"[Grade_cezio, Grade_mateus]",2


In [63]:
consensus

,Sentence,Grade_cezio,Grade_evandro,Grade_mateus,matched_grade,matched_by,match_count
0,A atividade econômica deverá se recuperar nos ...,1,1,1,1.0,"[Grade_cezio, Grade_evandro, Grade_mateus]",3
1,A carteira de operações realizadas com recurso...,-2,-3,-2,-2.0,"[Grade_cezio, Grade_mateus]",2
2,"A conjuntura atual, caracterizada por um estág...",1,0,1,1.0,"[Grade_cezio, Grade_mateus]",2
3,A decisão do Copom de reduzir a meta para a ta...,1,1,1,1.0,"[Grade_cezio, Grade_evandro, Grade_mateus]",3
4,A elevação da meta para a Taxa Selic nas duas ...,-2,0,-2,-2.0,"[Grade_cezio, Grade_mateus]",2
...,...,...,...,...,...,...,...
83,Progressos na estrutura dos mercados financeir...,1,1,1,1.0,"[Grade_cezio, Grade_evandro, Grade_mateus]",3
84,"Quanto às taxas de inflação, elas têm permanec...",-2,0,0,0.0,"[Grade_evandro, Grade_mateus]",2
85,Relativamente ao Índice Nacional de Preços ao ...,0,0,-1,0.0,"[Grade_cezio, Grade_evandro]",2
86,Segundo pesquisa do IBGE nas principais regiõe...,-2,-3,-2,-2.0,"[Grade_cezio, Grade_mateus]",2


In [64]:
# DataFrame com as linhas onde os 3 avaliadores discordaram (todos os grades diferentes)
three_way_disagree = evaluations_all[evaluations_all[grade_cols].nunique(axis=1) == 3].reset_index(drop=True)
print("three_way_disagree shape:", three_way_disagree.shape)
three_way_disagree.head()

three_way_disagree shape: (11, 4)


,Sentence,Grade_cezio,Grade_evandro,Grade_mateus
0,"A queda, próxima a 9%, atingiu os dois compone...",-1,-3,-2
1,"Considerada a série observada, o Nuci apresent...",0,-1,-2
2,"Dito de outra forma, taxas de inflação elevada...",-2,0,-1
3,"Em particular, a expectativa de uma redução no...",1,-1,-2
4,"Em particular, cabe à política monetária atuar...",-2,0,-1


In [70]:
out_path = base_dir / "three_way_disagree.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
three_way_disagree.to_csv(out_path, sep='|', index=False, encoding='utf-8')
print(f"Saved {len(three_way_disagree)} rows to {out_path}")

Saved 11 rows to inflação/data/anotated/complete/three_way_disagree.csv


In [66]:
three_way_disagree

,Sentence,Grade_cezio,Grade_evandro,Grade_mateus
0,"A queda, próxima a 9%, atingiu os dois compone...",-1,-3,-2
1,"Considerada a série observada, o Nuci apresent...",0,-1,-2
2,"Dito de outra forma, taxas de inflação elevada...",-2,0,-1
3,"Em particular, a expectativa de uma redução no...",1,-1,-2
4,"Em particular, cabe à política monetária atuar...",-2,0,-1
5,"Em relação à reunião anterior, as projeções de...",1,-1,0
6,"Entretanto, o Copom considerou que a terceira ...",1,-1,0
7,Já a inflação medida pelo Índice de Preços ao ...,1,0,-2
8,No cenário com trajetórias para as taxas Selic...,1,0,-2
9,"Para os setores de bens de consumo, bens de ca...",-1,-3,-2


In [67]:
# substituir -2 e -3 por 0 nas colunas de avaliação e recalcular consenso/discrepâncias
evaluations_all_mod = evaluations_all.copy()
evaluations_all_mod[grade_cols] = evaluations_all_mod[grade_cols].replace({-2: 0, -3: 0})

# recalcular info/consensus usando a função _match_info já definida
info_mod = evaluations_all_mod.apply(_match_info, axis=1)
consensus_mod = pd.concat([evaluations_all_mod, info_mod], axis=1)
consensus_mod = consensus_mod[consensus_mod['match_count'] >= 2].reset_index(drop=True)

# linhas onde os 3 avaliadores discordam (após a substituição)
three_way_disagree_mod = evaluations_all_mod[evaluations_all_mod[grade_cols].nunique(axis=1) == 3].reset_index(drop=True)

print("evaluations_all_mod shape:", evaluations_all_mod.shape)
print("consensus_mod shape:", consensus_mod.shape)
print("three_way_disagree_mod shape:", three_way_disagree_mod.shape)

# calcular Cohen's kappa entre pares após a substituição
cezio_mod = evaluations_all_mod[['Sentence', 'Grade_cezio']].rename(columns={'Grade_cezio': 'Grade'})
evandro_mod = evaluations_all_mod[['Sentence', 'Grade_evandro']].rename(columns={'Grade_evandro': 'Grade'})
mateus_mod = evaluations_all_mod[['Sentence', 'Grade_mateus']].rename(columns={'Grade_mateus': 'Grade'})

k_results = {}
k_results['cezio_vs_evandro'] = pairwise_kappa(cezio_mod, evandro_mod, name1='cezio', name2='evandro')
k_results['cezio_vs_mateus'] = pairwise_kappa(cezio_mod, mateus_mod, name1='cezio', name2='mateus')
k_results['evandro_vs_mateus'] = pairwise_kappa(evandro_mod, mateus_mod, name1='evandro', name2='mateus')

# resumo em DataFrame
summary_mod = []
for pair, res in k_results.items():
    if res is None:
        summary_mod.append({'pair': pair, 'kappa': None, 'n': 0})
    else:
        kappa, matched = res
        summary_mod.append({'pair': pair, 'kappa': float(kappa), 'n': len(matched)})

pd.DataFrame(summary_mod)

evaluations_all_mod shape: (99, 4)
consensus_mod shape: (95, 7)
three_way_disagree_mod shape: (4, 4)
cezio vs evandro: Cohen's kappa = 0.5125 (n = 99)
cezio vs mateus: Cohen's kappa = 0.4918 (n = 99)
evandro vs mateus: Cohen's kappa = 0.4604 (n = 99)


,pair,kappa,n
0,cezio_vs_evandro,0.512465,99
1,cezio_vs_mateus,0.491834,99
2,evandro_vs_mateus,0.460364,99


In [69]:
out_path = base_dir / "evaluations_all.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
evaluations_all.to_csv(out_path, sep='|', index=False, encoding='utf-8')
print(f"Saved {len(evaluations_all)} rows to {out_path}")

Saved 99 rows to inflação/data/anotated/complete/evaluations_all.csv


In [68]:
evaluations_all

,Sentence,Grade_cezio,Grade_evandro,Grade_mateus
0,A atividade econômica deverá se recuperar nos ...,1,1,1
1,A carteira de operações realizadas com recurso...,-2,-3,-2
2,"A conjuntura atual, caracterizada por um estág...",1,0,1
3,A decisão do Copom de reduzir a meta para a ta...,1,1,1
4,A elevação da meta para a Taxa Selic nas duas ...,-2,0,-2
...,...,...,...,...
94,Progressos na estrutura dos mercados financeir...,1,1,1
95,"Quanto às taxas de inflação, elas têm permanec...",-2,0,0
96,Relativamente ao Índice Nacional de Preços ao ...,0,0,-1
97,Segundo pesquisa do IBGE nas principais regiõe...,-2,-3,-2


# Pós conciliação

In [6]:
!pwd

/home/users/u7272942


In [8]:
# carregar all_post_conciliation.csv
file_name = "inflação/data/anotated/complete/all_post_conciliation.csv"
all_post_conciliation = pd.read_csv(file_name, sep=',', encoding='utf-8', engine='python')
all_post_conciliation.head()

Carregado inflação/data/anotated/complete/all_post_conciliation.csv — shape: (500, 7)


,cod,texto,cezio,evandro,mateus,discordancia total,conciliado
0,MDIwOTIwMDkudHh0,"No médio prazo, o risco advém dos efeitos, cum...",-1.0,-1.0,-2.0,NaN,-1.0
1,MzAxMTIwMTEudHh0,"Os preços livres aumentaram 7, 24% no período ...",-1.0,-1.0,-1.0,NaN,-1.0
2,MjExMDIwMDkudHh0,"Nesse contexto, após um período de flexibiliza...",-2.0,-1.0,-2.0,NaN,-2.0
3,MTcwNDIwMTMudHh0,"Considerada a série observada, o Nuci apresent...",-1.0,-1.0,-1.0,NaN,-1.0
4,MjAxMTIwMDIudHh0,Dados preliminares da Fecomércio mostram que o...,-1.0,-1.0,-2.0,NaN,-1.0


In [11]:
# calcular Cohen's kappa nas primeiras 220 linhas para cezio, evandro e mateus
subset = all_post_conciliation.iloc[:220]

pairs = [('cezio', 'evandro'), ('cezio', 'mateus'), ('evandro', 'mateus')]

results = []
for a, b in pairs:
    df_pair = subset[[a, b]].dropna()
    n = len(df_pair)
    if n == 0:
        print(f"Aviso: sem pares válidos para {a} vs {b} nas primeiras 220 linhas.")
        results.append({'pair': f'{a}_vs_{b}', 'kappa': None, 'n': 0})
        continue
    # converter para int quando possível (as notas estão como floats que representam inteiros)
    try:
        labels_a = df_pair[a].astype(int)
        labels_b = df_pair[b].astype(int)
    except Exception:
        labels_a = df_pair[a]
        labels_b = df_pair[b]
    k = cohen_kappa_score(labels_a, labels_b)
    print(f"{a} vs {b}: Cohen's kappa = {k:.4f} (n = {n})")
    results.append({'pair': f'{a}_vs_{b}', 'kappa': float(k), 'n': n})

pd.DataFrame(results)

cezio vs evandro: Cohen's kappa = 0.4555 (n = 220)
cezio vs mateus: Cohen's kappa = 0.4592 (n = 220)
evandro vs mateus: Cohen's kappa = 0.4204 (n = 220)


,pair,kappa,n
0,cezio_vs_evandro,0.455476,220
1,cezio_vs_mateus,0.459155,220
2,evandro_vs_mateus,0.420433,220


In [15]:
import pandas as pd

# substituir -2 e -3 por 0 nas colunas cezio, evandro e mateus nas primeiras 220 linhas
cols = ['cezio', 'evandro', 'mateus']
subset_220 = all_post_conciliation.iloc[:220].copy()
subset_220[cols] = subset_220[cols].replace({-2: 0, -3: 0})

# calcular Cohen's kappa para os pares
from sklearn.metrics import cohen_kappa_score  # já importado no notebook, mas seguro garantir aqui no novo contexto

pairs = [('cezio', 'evandro'), ('cezio', 'mateus'), ('evandro', 'mateus')]
results_220 = []
for a, b in pairs:
    df_pair = subset_220[[a, b]].dropna()
    n = len(df_pair)
    if n == 0:
        print(f"Aviso: sem pares válidos para {a} vs {b} nas primeiras 220 linhas.")
        results_220.append({'pair': f'{a}_vs_{b}', 'kappa': None, 'n': 0})
        continue
    try:
        labels_a = df_pair[a].astype(int)
        labels_b = df_pair[b].astype(int)
    except Exception:
        labels_a = df_pair[a]
        labels_b = df_pair[b]
    k = cohen_kappa_score(labels_a, labels_b)
    print(f"{a} vs {b}: Cohen's kappa = {k:.4f} (n = {n})")
    results_220.append({'pair': f'{a}_vs_{b}', 'kappa': float(k), 'n': n})

pd.DataFrame(results_220)

cezio vs evandro: Cohen's kappa = 0.5703 (n = 220)
cezio vs mateus: Cohen's kappa = 0.5655 (n = 220)
evandro vs mateus: Cohen's kappa = 0.5371 (n = 220)


,pair,kappa,n
0,cezio_vs_evandro,0.570327,220
1,cezio_vs_mateus,0.565543,220
2,evandro_vs_mateus,0.537147,220


# Dataset conciliado

In [16]:
# exportar os primeiros 220 registros da coluna 'conciliado'
conciliado_first220 = all_post_conciliation['conciliado'].iloc[:220].reset_index(drop=True)
df_out = conciliado_first220.to_frame()

out_path = Path(file_name).parent / "conciliado_first220.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(out_path, sep='|', index=False, encoding='utf-8')
print(f"Saved {len(df_out)} rows to {out_path}")

Saved 220 rows to inflação/data/anotated/complete/conciliado_first220.csv


In [17]:
# exportar os primeiros 350 registros da coluna 'cezio'
cezio_first350 = all_post_conciliation['cezio'].iloc[:350].reset_index(drop=True)
df_cezio = cezio_first350.to_frame()

out_path_cezio350 = Path(file_name).parent / "cezio_first350.csv"
out_path_cezio350.parent.mkdir(parents=True, exist_ok=True)
df_cezio.to_csv(out_path_cezio350, sep='|', index=False, encoding='utf-8')
print(f"Saved {len(df_cezio)} rows to {out_path_cezio350}")

Saved 350 rows to inflação/data/anotated/complete/cezio_first350.csv


In [19]:
from langchain.llms import Ollama

# Chamar modelo Ollama (gemma) via LangChain para "emular" a opinião do avaliador `cezio`.
# Reutiliza o DataFrame `all_post_conciliation` já presente no notebook.
# Observação: precisa do serviço Ollama rodando localmente.


# montar few-shot com alguns exemplos reais de cezio (filtrando nulos)
examples = all_post_conciliation[all_post_conciliation['cezio'].notna()].head(5)

few_shot = []
for _, r in examples.iterrows():
    grade = int(r['cezio'])
    texto = r['texto'].replace("\n", " ").strip()
    few_shot.append(f"Texto: {texto}\nCezio_grade: {grade}\n---")
few_shot_str = "\n".join(few_shot)

# texto de entrada a ser avaliado (troque por qualquer outra linha/texto)
input_text = all_post_conciliation.loc[0, 'texto']

prompt = f"""
Você é o avaliador "cezio". A seguir há exemplos de como "cezio" avaliou textos.
Formato de resposta exigido: apenas "GRADE||Justificação curta" (por exemplo: -1||Frase curta).
Grades possíveis: -2, -1, 0, 1, 2.

EXEMPLOS:
{few_shot_str}

AGORA AVALIE:
Texto: {input_text}

Responda no formato solicitado (GRADE||Justificação curta).
"""

# criar cliente Ollama e executar
llm = Ollama(model="gemma3")  # ajuste modelo/parametros conforme necessário
resp = llm(prompt)

print("Prompt enviado:")
print(prompt[:1000])  # mostrar início do prompt
print("\nResposta do modelo:")
print(resp)

Prompt enviado:

Você é o avaliador "cezio". A seguir há exemplos de como "cezio" avaliou textos.
Formato de resposta exigido: apenas "GRADE||Justificação curta" (por exemplo: -1||Frase curta).
Grades possíveis: -2, -1, 0, 1, 2.

EXEMPLOS:
Texto: No médio prazo, o risco advém dos efeitos, cumulativos e defasados, da distensão das condições financeiras e do impulso fiscal sobre a evolução da demanda doméstica, levando-se em conta a dinâmica do consumo e do investimento, em contexto de retomada da utilização dos fatores de produção.
Cezio_grade: -1
---
Texto: Os preços livres aumentaram 7, 24% no período de doze meses encerrado em outubro, ante 7, 79% em setembro, e os monitorados apresentaram aceleração no período, com a variação acumulada em doze meses atingindo 6, 34% ante 6, 17% no mês anterior.
Cezio_grade: -1
---
Texto: Nesse contexto, após um período de flexibilização agressiva, a política monetária, em um número significativo de países, parece ter entrado em fase de estabilidade 

In [ ]:
#substituir -2 e -3 por 0 nas colunas cezio, evandro e mateus e calcular para as primeiras 220 linhas


